# Notebook_busse

In de onderstaande cel worden alle modules die worden gebruikt in het bestand geïmporteerd.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data_test = pd.read_csv('test_c.csv', sep = ',')
data_train = pd.read_csv('train_c.csv', sep = ',')

In [4]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   p_id                      614 non-null    int64  
 1   Pregnancies               614 non-null    int64  
 2   Glucose                   614 non-null    int64  
 3   BloodPressure             614 non-null    int64  
 4   SkinThickness             614 non-null    int64  
 5   Insulin                   614 non-null    int64  
 6   BMI                       614 non-null    float64
 7   DiabetesPedigreeFunction  614 non-null    float64
 8   Age                       614 non-null    int64  
 9   Outcome                   614 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 48.1 KB


In [5]:
data_train.describe()

,p_id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,385.542345,3.809446,120.465798,68.903909,20.456026,81.250814,31.929642,0.468096,33.265472,0.342020
std,222.135103,3.375544,31.959619,19.933385,16.063105,116.947384,8.136791,0.329203,11.851427,0.474773
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,194.250000,1.000000,99.000000,62.000000,0.000000,0.000000,27.400000,0.238000,24.000000,0.000000
50%,380.500000,3.000000,115.000000,72.000000,23.000000,27.000000,32.150000,0.365500,29.000000,0.000000
75%,583.750000,6.000000,140.000000,80.000000,32.000000,130.000000,36.500000,0.628500,41.000000,1.000000
max,767.000000,15.000000,197.000000,114.000000,99.000000,744.000000,67.100000,2.420000,72.000000,1.000000


In [6]:
# Scheiden van de data in afhankelijke (y) en onafhankelijke (X) variabelen
X = data_train.drop('Outcome', axis=1)
y = data_train['Outcome']

In [7]:
print(len(data_train[data_train['Glucose'] == 0]))
print(len(data_train[data_train['BloodPressure'] == 0]))
print(len(data_train[data_train['SkinThickness'] == 0]))
print(len(data_train[data_train['Insulin'] == 0]))
print(len(data_train[data_train['BMI'] == 0]))

4
31
184
300
11


In [8]:
data_new = data_train.drop(data_train[data_train['BMI'] == 0].index)
data_new = data_new.drop(data_new[data_new['Glucose'] == 0].index)
data_new = data_new.drop(data_new[data_new['BloodPressure'] == 0].index)
data_new = data_new.drop('Insulin', axis=1)
data_new = data_new.drop('SkinThickness', axis=1)

print(len(data_new[data_new['Glucose'] == 0]))
print(len(data_new[data_new['BloodPressure'] == 0]))
print(len(data_new[data_new['BMI'] == 0]))
print(len(data_new))

0
0
0
575


In [9]:
data_new.describe()

,p_id,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age,Outcome
count,575.000000,575.00000,575.000000,575.000000,575.000000,575.000000,575.00000,575.000000
mean,387.502609,3.81913,121.582609,72.556522,32.551826,0.470833,33.38087,0.339130
std,222.159044,3.36284,30.743114,12.322885,6.934810,0.329679,11.82889,0.473826
min,0.000000,0.00000,56.000000,24.000000,18.200000,0.078000,21.00000,0.000000
25%,196.500000,1.00000,100.000000,64.000000,27.600000,0.239000,24.00000,0.000000
50%,383.000000,3.00000,116.000000,72.000000,32.400000,0.371000,29.00000,0.000000
75%,584.500000,6.00000,142.000000,80.000000,36.600000,0.635500,41.00000,1.000000
max,767.000000,15.00000,197.000000,114.000000,67.100000,2.420000,70.00000,1.000000


In [10]:
class Outliers:
    """
    A class to handle the outliers in a dataframe

    Attributes:
    --------------------
    df : pandas.DataFrame
        The dataframe of which you want to look for outliers

    Methods:
    --------------------
    check_skew(self, column, th=0.5)
        This function checks the skewness of a given
        column from the dataframe passed in the class

    def check_outliers(self, column, th=1.5, remove=False)
        This function checks a column for outliers if
        there is a normal distribution and possibly removes them.
        Checking the outliers is based on using the Inter Quartile Range.
    """

    def __init__(self, df):
        """
        The initiator, makes the passed df into a class df
        """

        self.df = df

    def check_skew(self, column, th=0.5):
        """
        This function checks the skewness of a given
        column from the dataframe passed in the class

        Parameters:
        --------------------
        self :
            The given name of the class

        column : str
            The column of the dataframe you want to check

        th : float (default = 0.5)
            The threshold of which a column is
            considered skewed or nearly symmetrical

        Returns:
        --------------------
        None :
            Instead returns two print statements and a graph
            detailing and showing the distribution of the column
        """

        # If-statement to check for int or float
        if all(isinstance(value, (int, float)) for value in self.df[column]):
            # Checking the skewness of the given column
            skewness = self.df[column].skew()

            # if-statement for the type of skewness
            if abs(skewness) <= th:
                skewness_type = "Symmetric"
            elif skewness > 0:
                skewness_type = "Right-skewed"
            else:
                skewness_type = "Left-skewed"

            # Printing the skewness type and value
            print(f"Skewness type: {skewness_type}")
            print(f"Skewness value: {skewness}")

            # Visualising the skewness
            plt.figure(figsize=(8, 6))
            sns.histplot(self.df[column], kde=True)
            plt.xlabel("Values")
            plt.ylabel("Frequency")
            plt.title(f"Distribution of {column}")
            plt.show()

        # raising an error when column isnt int or float
        else:
            raise ValueError(f"The column {column} is not "
                             "interger or float, so skewness "
                             "cannot be calculated")

    def check_outliers(self, column, th=1.5, remove=False):
        """
        This function checks a column for outliers if
        there is a normal distribution and possibly removes them.
        Checking the outliers is based on using the Inter Quartile Range.

        Parameters:
        --------------------
        self :
            The given name of the class

        column : str
            The name of the column you want to check

        th : float or int (default = 1.5)
            The threshold for the IQR, with th
            having the most common value of 1.5

        remove : bool (default = False)
            A boolean value that determines if
            the outliers will be removed or not

        Returns:
        --------------------
        None :
            Instead returns a print which shows the outliers
        """

        # Check if column is a string or a list
        if isinstance(column, str):
            # Process a single column
            columns = [column]
        elif isinstance(column, list) and \
                all(isinstance(col, str) for col in column):
            # Process each column in the list
            columns = column
        else:
            raise ValueError("The column parameter must be "
                             "a string or a list of strings.")

        # Create a list to store outliers for each column
        skipped_columns = []

        for col in columns:
            # If-statement to check for int or float
            if all(isinstance(value, (int, float)) for value in self.df[col]):
                # Making the quartile ranges
                q1 = np.percentile(self.df[col], 25)
                q3 = np.percentile(self.df[col], 75)
                iqr = q3 - q1

                # Making the upper and lower bounds for the values
                # to see if they are considered an outlier
                lower_bound = q1 - th * iqr
                upper_bound = q3 + th * iqr

                # Making a DataFrame where only the
                # outliers are contained within
                outliers = self.df[col][(self.df[col] <= lower_bound)
                                        | (self.df[col] >= upper_bound)]

                # If-statement to print out the amount of outliers
                if len(outliers) == 0:
                    print(f"No outliers were found in {col}.")
                else:
                    print(f"The following outliers are found in {col}:")
                    print(outliers)

                # If-statement for checking if the outliers will be removed.
                if remove is True:
                    # Remove outliers from the DataFrame
                    self.df = self.df[(self.df[col] >= lower_bound)
                                      & (self.df[col] <= upper_bound)]

            # Append the col into the skipped_columns list
            else:
                skipped_columns.append(col)
                continue

        # If columns are skipped, print the columns that were skipped
        if len(skipped_columns) > 0:
            print("The following columns were skipped "
                  f"due to non-numeric values: {skipped_columns}")

        return self.df

In [15]:
out = Outliers(data_new)
data_newer = out.check_outliers(list(data_new.columns), remove=True)

No outliers were found in p_id.
The following outliers are found in Pregnancies:
337    15
427    14
Name: Pregnancies, dtype: int64
No outliers were found in Glucose.
The following outliers are found in BloodPressure:
67     108
75     108
113    106
129    110
137    114
235     30
247     38
283    110
292    110
297    104
340     30
347    106
535    104
537     24
Name: BloodPressure, dtype: int64
The following outliers are found in BMI:
30     53.2
86     50.0
173    57.3
216    52.3
382    59.4
543    52.9
Name: BMI, dtype: float64
The following outliers are found in DiabetesPedigreeFunction:
46     1.390
108    1.292
123    1.268
127    1.353
134    1.258
146    1.224
150    1.251
155    1.282
184    1.224
203    1.600
236    1.699
352    2.329
416    1.441
453    1.461
470    1.893
479    1.698
504    2.137
555    1.400
Name: DiabetesPedigreeFunction, dtype: float64
The following outliers are found in Age:
31     67
287    70
396    68
465    67
564    69
Name: Age, dtype: in

In [16]:
data_newer

,p_id,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age,Outcome
0,668,6,98,58,34.0,0.430,43,0
1,324,2,112,75,35.7,0.148,21,0
2,624,2,108,64,30.8,0.158,21,0
3,690,8,107,80,24.6,0.856,34,0
4,473,7,136,90,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...
609,384,1,125,70,24.3,0.221,25,0
610,224,1,100,66,23.6,0.666,26,0
611,729,2,92,52,30.1,0.141,22,0
612,630,7,114,64,27.4,0.732,34,1


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

class Classificatie:
    """ 
    Een class die gebruikt wordt om een voorspellend model te maken. 
    
    Atributes:
    ------------------
    df : DataFrame
        Het dataframe waar het model op gebruikt wordt.
        
    target : str
        Naam van de kolom die voorspelt wordt.
        
    Methods:
    ----------------
    model_knn(self, neighbors = 5, weights = 'uniform', leaf_size = 30, p = 2)
        Een method die het aantal waardes, het aantal goed voorspelde waardes en de accuratie uitprint.
        Dit gebeurt op basis van het Machine Learning model genaamd KNearestNeighbors.
    
    find_k(self, high_range, ylim_low = None, ylim_high = None)
        Een method die een lijngrafiek laat zien met de accuratie van de test en train voorspellingen bij verschillend aantal neighbors.
    
    model_logreg(self, c_value = 1.0, penalty = 'l2')
        Een method die het classification report returned. Dit rapport wordt gemaakt door de
        classification_report method van SKLearn en bevat o.a. de Precision, Recall en Accuracy.
    
    grid_search(self, param_grid, knn_or_log = 'knn')
        Een method die de beste parameters en de beste Cross Validation score berekent en vervolgens toont.
    """
    
    def __init__(self, df, target):
        """
        De constructor van de class. Deze maakt X en y aan om deze vervolgens
        te splitten in een train en test set voor het modelleren.
        
        Parameters:
        -------------------
        df : DataFrame
            Het dataframe waar het model op gebruikt wordt.
        
        target : str
            Naam van de kolom die voorspelt wordt.
        """
        
        self.X = df.drop(target, axis = 1)
        self.y = df[target]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, random_state = 42)
    
    def model_logreg(self, c_value = 1.0, penalty = 'l2'):
        """
        Een method die het classification report returned. Dit rapport wordt gemaakt door de
        classification_report method van SKLearn en bevat o.a. de Precision, Recall en Accuracy.
        
        Parameters:
        -------------------
        c_value: float, default = 1.0
            Inverse van regularisatiekracht; moet een positieve floats zijn. Net als bij ondersteunende vectormachines, 
            specificeren kleinere waarden een sterkere regularisatie.
    
        penalty: str, default ='l2'
            Bepaald of er een penalty gebruikt wordt en welke er gekozen wordt.
            
        Returns:
        ------------------
        print : str
            Het rapport met daarin de waardes over hoe goed de voorspelling in elkaar zit.
        
        """
        
        #model aanmaken en fitten
        self.logreg = LogisticRegression(penalty = penalty, solver = 'liblinear', C = c_value, random_state = 42, max_iter = 500)
        self.logreg.fit(self.X_train, self.y_train)
        
        #berekenen y_hat en tonen calssificatie rapport
        self.y_hat = self.logreg.predict(self.X)
        self.report = classification_report(self.y, self.y_hat)
        
        return print('report:', self.report, sep='\n')

In [19]:
model = Classificatie(data_newer, 'Outcome')
model.model_logreg()

report:
              precision    recall  f1-score   support

           0       0.79      0.91      0.84       360
           1       0.72      0.50      0.59       173

    accuracy                           0.77       533
   macro avg       0.76      0.70      0.72       533
weighted avg       0.77      0.77      0.76       533

